# Open3Dで点群処理

In [1]:
import numpy as np
import open3d as o3

In [8]:
cloud_m = o3.read_point_cloud("./data/hammer_1.pcd")

# 汎用的に使える関数

In [9]:
def Centering( cloud_in ):
    """
    Centering()
    offset an input cloud to its centroid.
    
    input(s):
        cloud_in: point cloud to be centered
    output(s):
        cloud_off: 
    """
    np_m = np.asarray(cloud_in.points) 
    center = np.mean(np_m, axis=0)
    np_m[:] -= center
    
    cloud_off = o3.PointCloud()
    cloud_off.points = o3.Vector3dVector(np_m)
    
    return cloud_off

In [10]:
def Scaling( cloud_in, scale ):
    """
    Scaling()
    
    input(s):
        cloud_in: point cloud to be scaled.
        scale: scaling factor
    output(s):
        cloud_out: 
    """   
    cloud_np = np.asarray(cloud_in.points) 
    cloud_np *= scale
    cloud_out = o3.PointCloud()
    cloud_out.points = o3.Vector3dVector(cloud_np)
    
    return cloud_out

In [11]:
cloud_m_c = Centering(cloud_m)

In [12]:
cloud_m_c = Scaling(cloud_m_c, 0.001)

In [13]:
print(cloud_m_c)
print(np.asarray(cloud_m_c.points).shape)

PointCloud with 1057767 points.
(1057767, 3)


In [19]:
cloud_np = np.asarray(cloud_m_c.points)
cloud_min = np.min(cloud_np,axis=0)
print(cloud_min)

[-0.08521509 -0.09173449 -0.02257849]


In [8]:
voxel_size = 0.002
cloud_m_ds = o3.voxel_down_sample(cloud_m_c, voxel_size)
print(cloud_m_ds)
print(np.asarray(cloud_m_ds.points))

PointCloud with 13266 points.
[[-0.03124031 -0.02976083  0.02346314]
 [-0.01578999 -0.03845723  0.02255777]
 [-0.03380228 -0.02653974  0.02258453]
 ...
 [-0.02919985 -0.01938886  0.01141613]
 [-0.00041063 -0.03890944  0.01201567]
 [-0.03257511 -0.01696121  0.01201802]]


In [9]:
offset = np.array([0.03,0.13,0.88])

np_tmp = np.asarray(cloud_m_ds.points) 
np_tmp += offset
cloud_m_ds.points = o3.Vector3dVector(np_tmp)


In [11]:
print(cloud_m_ds.points[0])

[-0.00124031  0.10023917  0.90346314]


In [38]:
class Mapping():
    def __init__(self, camera_intrinsic_name, _w=640, _h=480, _d=400.0 ):
        self.camera_intrinsic = o3.read_pinhole_camera_intrinsic(camera_intrinsic_name)
        self.width = _w
        self.height = _h
        self.d = _d
        self.camera_intrinsic4x4 = np.identity(4)
        self.camera_intrinsic4x4[0,0] = self.camera_intrinsic.intrinsic_matrix[0,0]
        self.camera_intrinsic4x4[1,1] = self.camera_intrinsic.intrinsic_matrix[1,1]
        self.camera_intrinsic4x4[0,3] = self.camera_intrinsic.intrinsic_matrix[0,2]
        self.camera_intrinsic4x4[1,3] = self.camera_intrinsic.intrinsic_matrix[1,2]
        
    def showCameraIntrinsic(self):
        print(self.camera_intrinsic.intrinsic_matrix)
        print(self.camera_intrinsic4x4)

    def Cloud2Image( self, cloud_in ):
        
        img = np.zeros( [self.height, self.width], dtype=np.uint8 )
        
        cloud_np = np.asarray(cloud_in.points)
        cloud_np = cloud_np[:,:] / cloud_np[:,[2]]
        cloud_mapped = o3.PointCloud()
        cloud_mapped.points = o3.Vector3dVector(cloud_np)
        cloud_mapped.transform(self.camera_intrinsic4x4)
        print(np.asarray(cloud_mapped.points))
        
        for pix in cloud_mapped.points:
            if pix[0]<self.width and 0<pix[0] and pix[1]<self.height and 0<pix[1]:
                img[int(pix[1]),int(pix[0])] = 255
        
        return img

In [39]:
mapping = Mapping('realsense_intrinsic.json')

In [40]:
mapping.showCameraIntrinsic()

[[627.64233398   0.         327.36773682]
 [  0.         627.64233398 239.5690155 ]
 [  0.           0.           1.        ]]
[[627.64233398   0.           0.         327.36773682]
 [  0.         627.64233398   0.         239.5690155 ]
 [  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]]


In [41]:
img_mapped = mapping.Cloud2Image( cloud_m_ds )

[[326.50608505 309.20588429   1.        ]
 [337.24943306 303.22823149   1.        ]
 [324.72369704 311.51356591   1.        ]
 ...
 [327.93111911 317.44985351   1.        ]
 [348.1874811  303.66238367   1.        ]
 [325.55583824 319.1054454    1.        ]]


In [42]:
import cv2

In [43]:
print(img_mapped.shape)
cv2.imwrite('mapped.png',img_mapped)

(480, 640)


True

In [97]:
x = np.arange(15).reshape(5,3)
x

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [104]:
z = x[:,:] / x[:,[2]]

In [105]:
z

array([[0.        , 0.5       , 1.        ],
       [0.6       , 0.8       , 1.        ],
       [0.75      , 0.875     , 1.        ],
       [0.81818182, 0.90909091, 1.        ],
       [0.85714286, 0.92857143, 1.        ]])

In [112]:
intrin = np.identity(4)
zz = z * intrin

ValueError: operands could not be broadcast together with shapes (5,3) (4,4) 